In [38]:
%pip install -q surprise

Note: you may need to restart the kernel to use updated packages.


### Пример рекомендательной системы (User to Item)

In [44]:
import os
import joblib
import pandas as pd
import numpy as np
from surprise import Reader, Dataset, KNNBasic
from surprise.model_selection import cross_validate

In [40]:
BASE_DIR = r"C:\Users\andre\IdeaProjects\TyuiuDirectionsRecSys"

EXCELS_DIR = os.path.join(BASE_DIR, "data", "10 лет", "гигочат")

FILE_PATH = os.path.join(EXCELS_DIR, "2024.xlsx")

In [41]:
df = pd.read_excel(FILE_PATH)

In [42]:
COLUMNS = [
    "Пол",
    "Иностранное гражданство",
    "Ср. балл док-та об образовании",
    "Сумма баллов",
    "Обществознание",
    "мат",
    "Информатика и информационно–коммуникационные технологии (ИКТ)",
    "РУССКИЙ ЯЗЫК",
    "Физика",
    "Химия",
    "История",
    "Вид возмещения затрат",
    "Формирующее подр.",
    "Направление подготовки"
]

In [43]:
df = df[COLUMNS]
df = df[df["Вид возмещения затрат"] == "бюджет"]
df = df.drop("Вид возмещения затрат", axis=1)
df.head(5)

,Пол,Иностранное гражданство,Ср. балл док-та об образовании,Сумма баллов,Обществознание,мат,Информатика и информационно–коммуникационные технологии (ИКТ),РУССКИЙ ЯЗЫК,Физика,Химия,История,Формирующее подр.,Направление подготовки
0,Ж,NaN,5.000,247,NaN,84.0,NaN,89.0,64.0,NaN,NaN,Институт сервиса и отраслевого управления,21.03.02 Землеустройство и кадастры
1,Ж,NaN,5.000,234,NaN,NaN,NaN,68.0,NaN,NaN,NaN,Институт сервиса и отраслевого управления,43.03.00 Сервис и туризм
2,Ж,NaN,4.688,187,NaN,58.0,NaN,67.0,62.0,NaN,NaN,Институт промышленных технологий и инжиниринга,28.03.03 Наноматериалы
3,М,NaN,4.500,192,NaN,58.0,NaN,66.0,68.0,NaN,NaN,Институт сервиса и отраслевого управления,21.05.01 Прикладная геодезия
4,М,NaN,4.294,228,NaN,74.0,85.0,69.0,NaN,NaN,NaN,Институт геологии и нефтегазодобычи,15.03.04 Автоматизация технологических процесс...


In [45]:
df["Пол"] = df["Пол"].apply(lambda x: 1 if x == "М" else 0)
df = df.drop("Иностранное гражданство", axis=1)
df.head(3)

,Пол,Ср. балл док-та об образовании,Сумма баллов,Обществознание,мат,Информатика и информационно–коммуникационные технологии (ИКТ),РУССКИЙ ЯЗЫК,Физика,Химия,История,Формирующее подр.,Направление подготовки
0,0,5.000,247,NaN,84.0,NaN,89.0,64.0,NaN,NaN,Институт сервиса и отраслевого управления,21.03.02 Землеустройство и кадастры
1,0,5.000,234,NaN,NaN,NaN,68.0,NaN,NaN,NaN,Институт сервиса и отраслевого управления,43.03.00 Сервис и туризм
2,0,4.688,187,NaN,58.0,NaN,67.0,62.0,NaN,NaN,Институт промышленных технологий и инжиниринга,28.03.03 Наноматериалы


In [46]:
cols = [
    "Пол",
    "Ср. балл док-та об образовании",
    "Сумма баллов",
    "Обществознание",
    "мат",
    "Информатика и информационно–коммуникационные технологии (ИКТ)",
    "РУССКИЙ ЯЗЫК", 
    "Физика", 
    "Химия", 
    "История"
]

for col in cols:
    df[col].fillna(df[col].mean(), inplace=True)

for col in cols:
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

C:\Users\andre\AppData\Local\Temp\ipykernel_6976\2610187564.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)


In [52]:
ratings_df = pd.DataFrame(columns=['uid', 'iid', 'r'])

for idx, row in df.iterrows():
    for col in cols:
        new_row = pd.Series({'uid': idx + 1, 'iid': col, 'r': row[col]}, name=len(ratings_df))
        ratings_df = pd.concat([ratings_df, new_row.to_frame().T], ignore_index=True)

In [54]:
reader = Reader(rating_scale=(0, 100))

# Загружаем данные в формат surprise
dataset = Dataset.load_from_df(ratings_df, reader)

# Строим полный набор данных для тренировки
trainset = dataset.build_full_trainset()

In [56]:
sim_options = {'name': 'pearson_baseline'}
algo = KNNBasic(sim_options=sim_options)

# Кросс-валидация модели
cross_validate(algo, dataset, measures=['RMSE'], cv=5, verbose=False)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.18210861, 0.17878861, 0.18744279, 0.17908244, 0.17912596]),
 'fit_time': (1.691068172454834,
  1.704023838043213,
  1.6927423477172852,
  1.6549444198608398,
  1.6813693046569824),
 'test_time': (2.2850613594055176,
  2.372950792312622,
  2.441433906555176,
  2.4832358360290527,
  2.532362461090088)}

In [57]:
new_user_ratings = {
    "Пол": 1,
    "Ср. балл док-та об образовании": 4.2,
    "Сумма баллов": 227,
    'Обществознание': 0,
    'мат': 76,
    'Информатика и информационно–коммуникационные технологии (ИКТ)': 75,
    'РУССКИЙ ЯЗЫК': 64,
    'Физика': 0,
    'Химия': 0,
    'История': 0
}

# Приводим новые рейтинги к формату surprise
test_set = [(i+1, subject, rating) for i, (subject, rating) in enumerate(new_user_ratings.items())]

# Тестируем модель
predictions = algo.test(test_set)

# Рекомендуемое направление подготовки
best_prediction = max(predictions, key=lambda x: x.est)
print(f'Рекомендуемое направление подготовки: {best_prediction}')

Рекомендуемое направление подготовки: user: 2          item: Ср. балл док-та об образовании r_ui = 4.20   est = 0.97   {'actual_k': 40, 'was_impossible': False}


In [58]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Пример данных
data = {
    'Пол': ['Ж', 'Ж', 'Ж'],
    'Иностранное гражданство': [np.nan, np.nan, np.nan],
    'Ср. балл док-та об образовании': [5.000, 5.000, 4.688],
    'Сумма баллов': [247, 234, 187],
    'Обществознание': [np.nan, np.nan, np.nan],
    'мат': [84.0, np.nan, 58.0],
    'Информатика и информационно–коммуникационные технологии (ИКТ)': [np.nan, np.nan, np.nan],
    'РУССКИЙ ЯЗЫК': [89.0, 68.0, 67.0],
    'Физика': [64.0, np.nan, 62.0],
    'Химия': [np.nan, np.nan, np.nan],
    'История': [np.nan, np.nan, np.nan],
    'Формирующее подр.': ['Институт сервиса и отраслевого управления', 'Институт сервиса и отраслевого управления', 'Институт промышленных технологий и инжиниринга'],
    'Направление подготовки': ['21.03.02 Землеустройство и кадастры', '43.03.00 Сервис и туризм', '28.03.03 Наноматериалы']
}

df = pd.DataFrame(data)

# Заполнение пропущенных значений нулями
df_filled = df.fillna(0)

# Нормализация данных (если необходимо)
# Здесь можно использовать MinMaxScaler или StandardScaler из sklearn.preprocessing

# Создание профиля абитуриента
def create_student_profile(row):
    return row[['мат', 'РУССКИЙ ЯЗЫК', 'Физика']].values

# Создание профиля направления подготовки
def create_program_profile(df, program_name):
    program_df = df[df['Направление подготовки'] == program_name]
    return program_df[['мат', 'РУССКИЙ ЯЗЫК', 'Физика']].mean().values

# Пример профиля абитуриента
student_profile = create_student_profile(df_filled.iloc[0])

# Пример профиля направления подготовки
program_profiles = {}
for program in df['Направление подготовки'].unique():
    program_profiles[program] = create_program_profile(df_filled, program)

# Рекомендация направления подготовки
def recommend_program(student_profile, program_profiles):
    similarities = {}
    for program, profile in program_profiles.items():
        similarity = cosine_similarity([student_profile], [profile])[0][0]
        similarities[program] = similarity
    return max(similarities, key=similarities.get)

# Рекомендация для абитуриента
recommended_program = recommend_program(student_profile, program_profiles)
print(f"Рекомендуемое направление подготовки: {recommended_program}")

Рекомендуемое направление подготовки: 21.03.02 Землеустройство и кадастры
